## 1. 사용할 패키지 불러오기

In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torch import nn
import glob

## 2. 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

### (0) 압축 풀기

In [2]:
%cd /content/data

/content/data


In [4]:
!unzip -qq "/content/gdrive/MyDrive/Data_YW.zip"
!unzip -qq "/content/data/OfficeImageFinal.zip"

/content/data


### (1) 데이터 경로 정의

In [3]:
xlsx_path = 'Office_Image_Aesthetic_Final.xlsx'
image_path = './OfficeImageFinal/'

### (2) 평가 정보가 들어있는 xlsx 파일 불러오기

In [4]:
score = pd.read_excel(xlsx_path)
score.columns = ['no.', 'No.', 'building', 'address', '평균', '중앙값', 'Final1', 'Final2', 'Final3']
score = score.iloc[1:, :].reset_index(drop = True)
score.head()

,no.,No.,building,address,평균,중앙값,Final1,Final2,Final3
0,1.0,1.0,GFC(강남파이낸스센터),서울특별시 강남구 역삼동 737,3.333333,4,4,2,4
1,2.0,3.0,포스코센터,서울특별시 강남구 대치동 892,3.666667,4,4,2,5
2,3.0,4.0,"한화생명63빌딩(구,대한생명63빌딩)",서울특별시 영등포구 여의도동 60,3.333333,3,3,2,5
3,4.0,5.0,하이브랜드,서울특별시 서초구 양재동 215,3.666667,3,5,3,3
4,5.0,6.0,LG트윈타워,서울특별시 영등포구 여의도동 20,3,3,2,3,4


### (3) 사용할 Score 정의하기 [평균, 중앙값, Final1, Final2, Final3]

In [5]:
score_column = "평균"
label = score[score_column]

### (4) 이미지 파일 리스트 불러오기

In [6]:
filelist = glob.glob(image_path + '/*.png')

### (5) 각 이미지 번호에 해당하는 라벨 값 할당하기 (-로 split할 때 첫번째부분이 이미지번호)

In [8]:
label = [label[int(file.split('/')[-1].split('-')[0]) - 1] for file in filelist]

## 3. Pytorch Datagenerator 생성

### (1) CNN 모델의 Input size인 (224,224,3) 으로 Image Resize

In [10]:
transform_dict = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

### (2) Torch Dataset 생성

In [11]:
class ConstructDataset(Dataset):
    """
    Construct pytorch Dataset from file list.

    Parameters
    ----------
    file_list : list
        image file list
    target_list : list
        target list
    phase : str
        train phase. (Default: 'train')

    Returns
    --------
    pytorch Dataset
    """

    def __init__(self, file_list, labels, phase = 'train'):
        self.file_list = file_list
        self.labels = labels
        self.phase = phase

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        file_name = self.file_list[index]
        target_label = self.labels[index]
        image = cv2.imread(file_name)
        image = transform_dict[self.phase](image)
        return {'image': image, 'target': target_label}

### (3) Pytorch Datagenerator 생성

In [12]:
class dataset_generator(object):
    """
    Construct pytorch DataLoader from file list.

    Parameters
    ----------
    file_list : list
        image file list
    target_list : list
        target list
    batch_size : int
        batch size. (Default: 16)
    phase : str
        train phase. (Default: 'train')
    train_valid_split : bool
        whether to split data with train and validation. (Default: False)  
    valid_ratio : float
        validation ratio. (Default: 0.2) 
    random_seed : int
        random seed number (Default: 1004)  

    Returns
    --------
    pytorch DataLoader
    """
    def __init__(self, file_list, target_list, batch_size = 16, phase = 'train', train_valid_split = False, valid_ratio = 0.2, random_seed = 1004):
        self.file_list = file_list
        self.target_list = target_list
        self.batch_size = batch_size
        self.phase = phase
        self.train_valid_split = train_valid_split
        self.valid_ratio = valid_ratio
        self.random_seed = random_seed

    def dataloader(self):

        if self.phase == 'train':
            if self.train_valid_split:
                train_file_list, valid_file_list, train_target_list, valid_target_list = train_test_split(self.file_list, 
                                                                                        self.target_list, 
                                                                                        test_size=self.valid_ratio, 
                                                                                        random_state=self.random_seed)
                
                train_dataset = ConstructDataset(train_file_list, train_target_list, phase = 'train')
                valid_dataset = ConstructDataset(valid_file_list, valid_target_list, phase = 'valid')

                return dict({'train': DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True),
                            'valid': DataLoader(valid_dataset, batch_size=self.batch_size, shuffle=False)})


            else:
                train_dataset = ConstructDataset(self.file_list, self.target_list, phase = 'train')
                return dict({'train': DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)})
        
        else:
            test_dataset = ConstructDataset(self.file_list, self.target_list, phase = self.phase)
            return dict({'test': DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)})

In [13]:
batch_size = 4

In [14]:
image_dataset_generator = dataset_generator(filelist, label, batch_size=batch_size, phase= 'train', train_valid_split= True, valid_ratio=0.2, random_seed=1004)
dataloader = image_dataset_generator.dataloader()

## 4. Pytorch Pretrained Model 불러오기

사용 가능 모델: ['Efficientnet_b0 ~ b7', 'resnet', 'mobilenetv2']

In [73]:
model = models.efficientnet_b7(pretrained='imagenet')
# model = models.resnet50(pretrained='imagenet')
# model = models.MobileNetV2(pretrained='imagenet')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 5. Embedding Layer와 Prediction Layer 추가하기

### (1) Embedding 차원 정의

In [74]:
embedding_dimension = 64

### (2) Embedding Layer

In [75]:
embedding_part = nn.Sequential(nn.Sequential(nn.Linear(1000, embedding_dimension, bias=True), 
                               nn.ReLU()))

### (3) Prediction Layer

In [76]:
prediction_part = nn.Sequential(nn.Sequential(nn.Linear(embedding_dimension, 1, bias=True)))

### (4) 합치기

In [77]:
embedding_model = nn.Sequential(model, embedding_part, prediction_part)

## 6. Loss 및 Optimizer

In [78]:
learning_rate = 0.001

In [79]:
from torch.optim import Adam
loss_fun = nn.MSELoss()
optimizer = Adam(params = embedding_model.parameters(),
                        lr=learning_rate)

## 7. Train

In [80]:
epochs = 3

In [81]:
# gpu setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
embedding_model.to(device)


min_valid_loss = np.inf

# training
        
for e in range(epochs):
    train_loss = 0.0
    embedding_model.train()     # Optional when not using Model Specific layer
    for data in dataloader['train']:
        if torch.cuda.is_available():
            images, labels = data['image'].float().to(device), data['target'].float().to(device)
        
        optimizer.zero_grad()
        target = embedding_model(images)
        loss = loss_fun(target, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
            
    valid_loss = 0.0
    embedding_model.eval()     # Optional when not using Model Specific layer
    for data in dataloader['valid']:
        if torch.cuda.is_available():
            data, labels = data['image'].float().to(device), data['target'].float().to(device)
        
        target = embedding_model(data)
        loss = loss_fun(target, labels)
        valid_loss = loss.item() * data.size(0)
        

    print('Epoch {} \t Training Loss: {}, Validation Loss: {}'.format(e+1, train_loss / len(dataloader['train']), valid_loss / len(dataloader['valid'])))

    # Saving Best model (valid_loss 최소)
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        torch.save(embedding_model, 'Best_model.pt')


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 	 Training Loss: 1.0844447959721442, Validation Loss: 0.010094883830048317
Validation Loss Decreased(inf--->0.434080) 	 Saving The Model
Epoch 2 	 Training Loss: 0.4602694897388918, Validation Loss: 0.00013135659486748452
Validation Loss Decreased(0.434080--->0.005648) 	 Saving The Model
Epoch 3 	 Training Loss: 0.5494071433331663, Validation Loss: 0.014559921830199486


## 8. Load Best Model

In [33]:
prediction_model = torch.load('Best_model.pt')

## 9. Observe Embedding Feature and Score

### (1) Test Image 불러오기

In [29]:
image = '/content/data/OfficeImageFinal/1-1-kakao-face-2004.png'
true_label = label[int(image.split('/')[-1].split('-')[0]) - 1]
image = cv2.imread(image)
image = transform_dict['test'](image)
image = image.reshape([1, 3, 224, 224]).float().to(device)

### (2) Embedding Part 불러오기

In [63]:
embedding_model = prediction_model[:-1]

### (3) Observe Embedding Feature

In [68]:
embedding_model(image)

tensor([[0.0000, 3.4533, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.6599, 0.0000,
         0.0000, 2.3546, 0.0000, 1.3891, 3.7333, 1.4503, 1.8853, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4586, 3.0327, 0.0000, 0.0000,
         0.3753, 0.0000, 0.0000, 0.0000, 0.0000, 2.8689, 0.0000, 0.4947, 2.4318,
         0.0000, 0.0000, 2.3989, 0.7021, 0.0000, 0.7264, 0.0000, 0.0000, 0.0000,
         3.5740, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 2.2233, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         2.2297]], device='cuda:0', grad_fn=<ReluBackward0>)

### (4) Prediction of Score

In [70]:
print('Ground Truth: {}'.format(true_label))
print('Prediction: {}'.format(prediction_model(image).item()))

Ground Truth: 2.6666666666666665
Ground Truth: 2.9643917083740234
